In [1]:
import pandas as pd
train_df = pd.read_csv('/home/etaylor/code_projects/dubby/data/inbar_train_df.csv')
test_df = pd.read_csv('/home/etaylor/code_projects/dubby/data/inbar_test_df.csv')

In [2]:
import sys
# The path you want to add  
path_to_add = r'/home/etaylor/code_projects/dubby'  
# Add the directory to sys.path  
sys.path.insert(0, path_to_add)

In [3]:
print(len(f"len train_df = {train_df}"))
print(len(f"len test_df = {test_df}"))

3933
3847


In [4]:
train_df.head()

,Unnamed: 0,id,title,post,comment,father_comment,conversation_id,timestamp,meta.success
0,4,t1_cnhuxye,CMV:Essential Oils are bullshit,My wife has recently gotten deeply involved/ob...,[This answer in /r/askscience does a pretty go...,33 My wife has recently gotten deeply invol...,t3_2ro0ti,1.420706e+09,1.0
1,5,t1_cni1m79,CMV:Essential Oils are bullshit,My wife has recently gotten deeply involved/ob...,&amp;#8710; While my view may not have shifted...,36 [This answer in /r/askscience does a pre...,t3_2ro0ti,1.420719e+09,1.0
2,6,t1_cnhowj2,CMV:Essential Oils are bullshit,My wife has recently gotten deeply involved/ob...,Found a nice little [article](http://pediatric...,33 My wife has recently gotten deeply invol...,t3_2ro0ti,1.420696e+09,0.0
3,7,t1_cniedbg,CMV:Essential Oils are bullshit,My wife has recently gotten deeply involved/ob...,Debunking articles typically aren't peer-revie...,61 This article would not be accepted by an...,t3_2ro0ti,1.420758e+09,0.0
4,8,t1_cnhlmdo,CMV: I think the Paris shooting makes a good c...,Thinking about today's news: [BBC](http://www....,&gt;PS: I am not here to talk about USA and it...,64 Thinking about today's news: [BBC](http:...,t3_2rnr30,1.420691e+09,0.0


In [6]:
import pandas as pd
from src import classification_prompts


combinations = [
    ['title', 'comment'],
    ['title', 'post', 'comment'],
    ['title', 'post', 'comment', 'father_comment']
]

def process_data_for_classification(df, features):
    df = df.dropna(subset=features + ['meta.success'])  # Drop rows with NaN in any of the specified features or 'meta.success'
    processed_df = pd.DataFrame()

    def format_text(row):
        parts = [f"{feature}: {row[feature]}" for feature in features]
        return ' [SEP] '.join(parts)

    processed_df['text'] = df.apply(lambda x: format_text(x), axis=1)
    processed_df['label'] = df['meta.success'].astype(int)
    return processed_df

In [50]:
processed_train_df = process_data_for_classification(train_df, combinations[0])

In [7]:
import os 

folder_path = "/home/etaylor/code_projects/dubby/data/updated_data"

# Process and save for each combination
for combo in combinations:
    processed_train_df = process_data_for_classification(train_df, combo)
    processed_test_df = process_data_for_classification(test_df, combo)
    
    train_filename = f"train_{'_'.join(combo)}.csv"
    test_filename = f"test_{'_'.join(combo)}.csv"
    
    processed_train_df.to_csv(os.path.join(folder_path, train_filename), index=False)
    processed_test_df.to_csv(os.path.join(folder_path, test_filename), index=False)

# List the files saved
os.listdir(folder_path)

['train_title_comment.csv',
 'test_title_comment.csv',
 'train_title_post_comment.csv',
 'test_title_post_comment.csv',
 'train_title_post_comment_father_comment.csv',
 'test_title_post_comment_father_comment.csv']

In [14]:
combo = ['comment']

processed_train_df = process_data_for_classification(train_df, combo)
processed_test_df = process_data_for_classification(test_df, combo)

train_filename = f"train_{'_'.join(combo)}.csv"
test_filename = f"test_{'_'.join(combo)}.csv"

processed_train_df.to_csv(os.path.join(folder_path, train_filename), index=False)
processed_test_df.to_csv(os.path.join(folder_path, test_filename), index=False)

In [12]:
train_title_comment_df = pd.read_csv(os.path.join(folder_path, 'test_title_post_comment_father_comment.csv'))
train_title_comment_df.head()

,text,label
0,title: CMV: Anything that is man-made is natur...,1
1,title: CMV: Anything that is man-made is natur...,1
2,title: CMV: Anything that is man-made is natur...,1
3,title: CMV: Anything that is man-made is natur...,0
4,title: CMV: Veganism and vegetarianism are not...,1


In [13]:
from pprint import pprint
pprint(train_title_comment_df['text'][1])

("title: CMV: Anything that is man-made is natural. [SEP] post: I can't "
 'remember the topic that spurred this discussion, but a friend and I were '
 'debating whether man-made things were natural. He took the position that '
 'they are unnatural. \n'
 '\n'
 'He cited this definition by Merriam-Webster:  existing in nature and not '
 'made or caused by people : coming from nature '
 '(http://www.merriam-webster.com/dictionary/natural) as his basis for the '
 'distinction for natural vs. unnatural.\n'
 '\n'
 'However, I respectfully disagree with his position and furthermore that '
 "definition of natural. People arise from nature. Humankind's capacity to "
 'create, problem-solve, analyze, rationalize, and build also come from '
 'natural processes. How are the things we create unnatural? It is only '
 'through natural occurrences that we have this ability, why is it that we '
 'would give the credit of these things solely to man, as opposed to nature? '
 'We are not separate from na